In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import joblib
import pandas as pd
import numpy as np
import faiss
sys.path.append('../')

os.environ['TRANSFORMERS_CACHE'] = '/data/users/kartik/hf_cache/'
os.environ['CUDA_VISIBLE_DEVICES']='0'

from prod_code.get_predictions import GetPredictions



2022-08-09 14:26:58.117744: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Seed set to 42


In [2]:
load_path='/data/users/kartik/Dictionary_and_Word_Embeddings/checkpoints/xlm-roberta-large_bert-base-uncased_0'
# load_path='/data/users/kartik/Dictionary_and_Word_Embeddings/checkpoints/xlm-roberta-large_xlm-roberta-large_0'


In [6]:
from utils.faiss_utils import create_and_store_index, get_top_n_accuracy
# from trainer.params import get_roberta_params, get_xlm_params
import json

data_files = ['../data/clean_data/train.joblib','../data/clean_data/val.joblib','../data/clean_data/test.joblib',
            '../data/clean_data/de_wiki_muse.joblib'][1:-1]

params = json.load(open(load_path+'/params.json'))
create_and_store_index(data_files, params['emb_type'], params['knn_measure'], index_name='codwoe_VT')


Loading Data for Indexing...
loading files: ['../data/clean_data/val.joblib', '../data/clean_data/test.joblib']
Creating FAISS Index...
            word                                              gloss  \
0           1-up  In a game with multiple players , an indicator...   
1          10-20                                         Location .   
2            10s    Tens , written as a number but in plural form .   
3            2.0  A second major version of something or , figur...   
4     360-degree                Able to turn freely about an axis .   
...          ...                                                ...   
8945      zinger  A very rapidly moving object , especially one ...   
8946      zinger  An event that when experienced leaves the witn...   
8947      zinger  An outstanding , energetic and surprising thin...   
8948      zinger  Something to eat or drink that is tart and sti...   
8949         zzz                                To sleep or snore .   

           

KeyError: 'bert-base-uncased'

In [4]:
params

{'model_checkpoint': 'xlm-roberta-large',
 'source_column': 'gloss',
 'max_len': 150,
 'batch_size': 16,
 'dropout': 0.1,
 'learning_rate': 0.0001,
 'num_epochs': 150,
 'early_stopping_limit': 5,
 'device': 'cuda:0',
 'loss_fn_name': 'cosine',
 'emb_type': 'xlm-roberta-large',
 'use_adapters': True,
 'resume_from_checkpoint': False,
 'output_size': 1024,
 'knn_measure': 'cosine',
 'num_warmup_steps': 300,
 'extra_data': False}

In [13]:
# val

In [39]:
val = joblib.load('../data/clean_data/val.joblib')
# val = joblib.load('../data/clean_data/de_wiki_muse.joblib')
ranks, nearest = getpred.get_predictions(val, NEAREST_NEIGHBOUR_NUM = 1000, pred_batch = 64) #

  0%|          | 0/66 [00:00<?, ?it/s]

Average Rank @1000 : 158.88196255036738
Median Rank @1000 : 16.0
Rank Variance @1000 : 296.1288889106231
Top 1 accuracy is 0.168
Top 2 accuracy is 0.242
Top 5 accuracy is 0.353
Top 10 accuracy is 0.443
Top 50 accuracy is 0.645
Top 100 accuracy is 0.723


In [22]:
getpred.get_prediction_on_sentence("आराम की स्थिति में।")

['sleepily',
 'gentle',
 'heavy-hearted',
 'mellow',
 'slackish',
 'poise',
 'weak',
 'footbath',
 'hunch',
 'pleasurable',
 'passionless',
 'sideboard',
 'sunbathe',
 'prescient',
 'worn-out',
 'up',
 'linger',
 'knees-up',
 'brazen',
 'languish',
 'cabined',
 'angst',
 'indulge',
 'cavort',
 'dormancy',
 'aimlessly',
 'pleasant',
 'recline',
 'drift',
 'swimmy',
 'snoring',
 'companionably',
 'skiddly',
 'sedentary',
 'tensen',
 'swimmingly',
 'belt',
 'noiseless',
 'ankle-deep',
 'resolute',
 'stiffness',
 'floody',
 'hard',
 'dampness',
 'roughcast',
 'galvanic',
 'thwart']